In [53]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


In [6]:
url = 'https://www.alibaba.ir/gtours/all'
driver = webdriver.Safari()
driver.get(url)
time.sleep(5)

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Allow time for more content to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


In [8]:
# Get the page source after scrolling
response = driver.page_source
# Parse the page source with Beautiful Soup
bs = BeautifulSoup(response , 'html.parser')

In [9]:
all_tours = bs.find('div', attrs = {'class' : 'a-card gtour-available-card mb-4'})

In [228]:
title_list = []
destination_list = []
length_list = []
minimum_price_list = []
start_date_list = []
grade_of_the_hotel_list = []
meals_list = []
tours_detail = bs.find_all('div' , attrs = {'class' : 'flex py-4 px-2 mt-3'})


title = bs.find_all('div' , attrs = {'class' : 'text-headline-lg'})
destination = bs.find_all('div' , attrs ={'class' : 'text-body-lg mt-3'})
length = bs.find_all('div' , attrs = {'class' : 'text-body-lg mt-3'})
minimum_price = bs.find_all('span', attrs ={'tdata-v-7ffdd062': ''})
start_date = bs.find_all('div', attrs = {'class' : 'flex py-4 px-2 mt-3'})
grade_of_the_hotel = bs.find_all('div' ,attrs = {'class' : 'flex py-4 px-2 mt-3'})
meals = bs.find_all('div',attrs = {'class': 'flex py-4 px-2 mt-3'})
       
  

    

each_list_length = [
    len(title),
    len(destination),
    len(length),
    len(minimum_price),
    len(start_date),
    len(grade_of_the_hotel),
    len(meals),
    ]
grade_divs = grade_of_the_hotel[i].find_all('div')
for i in range(0,each_list_length[0] ):
    title_list.append(title[i].get_text())
    destination_list.append(destination[i].find('span').get_text() if destination[i] else 'N/A')
    length_list.append(length[i].find_all('span')[2].get_text() if length[i] else 'N/A')
    minimum_price_list.append(minimum_price[i].find('strong') if minimum_price[i] else 'N/A')
    start_date_list.append(start_date[i].find_all('div')[0].find_all('div')[1].get_text() if start_date[i] else 'N/A')
    grade_of_the_hotel_list.append(grade_of_the_hotel[i].find_all('div')[3].find_all('div')[1].get_text() if grade_of_the_hotel[i]  else 'N/A')
    meals_list.append(meals[i].find_all('div')[12].find_all('div')[1].get_text() if meals[i] else 'N/A')

    


In [220]:
each_list_length

[156, 156, 156, 2357, 156, 156, 156]

In [229]:
df = pd.DataFrame(list(zip(title_list,destination_list,length_list,minimum_price_list,start_date_list,grade_of_the_hotel_list,meals_list)), 
                  columns = ['Title','Destination','Length','MinimunPrice','StartDate','HotelGrade','Meals'])

In [230]:
df

,Title,Destination,Length,MinimunPrice,StartDate,HotelGrade,Meals
0,تور مثلث طلایی هند (هتل 4 ستاره),هند,8 روز و 7 شب,None,17 مهر,7 شب اقامت هتل 4 ستاره,۷ وعده صبحانه، ۳ وعده ناهار
1,تور ژاپن (توکیو - اوساکا),ژاپن,9 روز و 8 شب,None,23 آبان,هتل 4 ستاره,6 وعده صبحانه
2,تور اسپانیا 5 روزه,"اسپانیا,اروپا,بارسلونا",5 روز و 4 شب,None,12 آذر,هتل 4 ستاره,همه صبحانه‌ها
3,تور امارات - قطر (8 روزه),"قطر,دبی",8 روز و 7 شب,None,15 آذر,هتل 4 ستاره,صبحانه به تعداد شب های اقامت
4,نمایشگاه گوانگژو چین (کانتون),"چین,گوانگژو",9 روز و 8 شب,None,24 مهر,هتل 4 ستاره,صبحانه به تعداد شب های اقامت
...,...,...,...,...,...,...,...
151,کروز خلیج فارس (امارات، قطر، بحرین),دبی,8 روز و 7 شب,"[17,000,000]",3 آذر,کشتی کروز MSC Euribia,تمام وعده‌های غذایی سفر
152,تور مالزی و سنگاپور (9 روزه - هتل 5 ستاره),"مالزی,سنگاپور",9 روز و 8 شب,None,6 آذر,هتل 5 ستاره,صبحانه به تعداد شب های اقامت
153,تور بارسلون و مادرید (8 روزه),اسپانیا,8 روز و 7 شب,"[1,000]",12 آذر,هتل 4 ستاره,صبحانه به تعداد شب های اقامت
154,تور سوئیس - امارات,"دبی,سوئیس",8 روز و 7 شب,None,16 آذر,هتل 4 و 5 ستاره,صبحانه به تعداد شب های اقامت


AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?